# Basic Usage of Causal Inference Library

This notebook demonstrates the basic usage of the causal inference library for marketing applications.

## Overview

We'll cover:
1. Data preparation and validation
2. Basic causal inference with G-computation
3. Inverse Probability Weighting (IPW)
4. Augmented IPW (AIPW) - doubly robust estimation
5. Diagnostics and validation

In [ ]:
import sys
import os

# Add the project root to Python path for development mode compatibility
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, os.path.join(project_root, 'libs'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import causal inference components
from causal_inference.core.base import TreatmentData, OutcomeData, CovariateData
from causal_inference.estimators.g_computation import GComputationEstimator
from causal_inference.estimators.ipw import IPWEstimator
from causal_inference.estimators.aipw import AIPWEstimator
from causal_inference.data.synthetic import SyntheticDataGenerator
from causal_inference.data.validation import validate_causal_data
from causal_inference.diagnostics.balance import check_covariate_balance
from causal_inference.diagnostics.overlap import assess_positivity

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")

## 1. Data Generation and Preparation

Let's start by generating synthetic data that mimics a marketing campaign scenario.

In [ ]:
# Generate synthetic marketing campaign data
generator = SyntheticDataGenerator(random_state=42)

# Generate data for email campaign effectiveness study
treatment, outcome, covariates = generator.generate_linear_binary_treatment(
    n_samples=1000,
    n_confounders=4,
    treatment_effect=2.5,  # True ATE of $2.50 increase in purchase amount
    confounding_strength=0.5
)

print(f"Generated data with {len(treatment.values)} samples")
print(f"Treatment type: {treatment.treatment_type}")
print(f"Outcome type: {outcome.outcome_type}")
print(f"Number of covariates: {covariates.values.shape[1]}")
print(f"Treatment distribution: {np.bincount(treatment.values)}")

In [ ]:
# Examine the data
data_df = pd.DataFrame({
    'treatment': treatment.values,
    'outcome': outcome.values,
    **{f'covariate_{i}': covariates.values.iloc[:, i] for i in range(covariates.values.shape[1])}
})

print("Data summary:")
print(data_df.describe())

# Visualize treatment and outcome distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Treatment distribution
axes[0].bar(['Control', 'Treatment'], np.bincount(treatment.values))
axes[0].set_title('Treatment Distribution')
axes[0].set_ylabel('Count')

# Outcome by treatment
sns.boxplot(data=data_df, x='treatment', y='outcome', ax=axes[1])
axes[1].set_title('Outcome by Treatment')
axes[1].set_xlabel('Treatment Group')

# Covariate correlation
covariate_corr = covariates.values.corr()
sns.heatmap(covariate_corr, annot=True, ax=axes[2])
axes[2].set_title('Covariate Correlations')

plt.tight_layout()
plt.show()

## 2. Data Validation

Before running causal inference, let's validate our data to check for potential issues.

In [ ]:
# Validate the causal data
validation_result = validate_causal_data(
    treatment=treatment,
    outcome=outcome,
    covariates=covariates,
    check_overlap=True
)

print("Data validation completed successfully!")
print(f"Sample size: {len(treatment.values)}")
print(f"Treatment variation: {len(np.unique(treatment.values))} unique values")
print(f"No missing values: {not pd.isnull(covariates.values).any().any()}")

## 3. Causal Inference with G-Computation

G-computation (standardization) estimates the ATE by predicting potential outcomes under different treatment scenarios.

In [ ]:
# Initialize G-computation estimator
g_comp = GComputationEstimator(
    outcome_model='linear',  # Linear regression for continuous outcomes
    bootstrap_samples=100   # For confidence intervals
)

# Fit the estimator
print("Fitting G-computation estimator...")
g_comp.fit(treatment, outcome, covariates)

# Estimate the Average Treatment Effect (ATE)
g_comp_effect = g_comp.estimate_ate()

print("\n=== G-Computation Results ===")
print(f"Estimated ATE: ${g_comp_effect.ate:.2f}")
print(f"95% Confidence Interval: [${g_comp_effect.confidence_interval[0]:.2f}, ${g_comp_effect.confidence_interval[1]:.2f}]")
print(f"Effect is significant: {g_comp_effect.is_significant}")
print(f"True ATE: $2.50")

# Get detailed summary
print("\nDetailed Summary:")
print(g_comp.summary())

## 4. Inverse Probability Weighting (IPW)

IPW estimates the ATE by reweighting observations based on the propensity score (probability of receiving treatment).

In [ ]:
# Initialize IPW estimator
ipw = IPWEstimator(
    propensity_model='logistic',  # Logistic regression for binary treatment
    stabilize_weights=True,       # Use stabilized weights
    weight_truncation='percentile',  # Truncate extreme weights
    bootstrap_samples=100
)

# Fit the estimator
print("Fitting IPW estimator...")
ipw.fit(treatment, outcome, covariates)

# Estimate ATE
ipw_effect = ipw.estimate_ate()

print("\n=== IPW Results ===")
print(f"Estimated ATE: ${ipw_effect.ate:.2f}")
print(f"95% Confidence Interval: [${ipw_effect.confidence_interval[0]:.2f}, ${ipw_effect.confidence_interval[1]:.2f}]")
print(f"Effect is significant: {ipw_effect.is_significant}")

# Check weight distribution
weights = ipw._compute_weights()
print(f"\nWeight statistics:")
print(f"Mean weight: {np.mean(weights):.3f}")
print(f"Weight range: [{np.min(weights):.3f}, {np.max(weights):.3f}]")
print(f"Weight variance: {np.var(weights):.3f}")

# Plot weight distribution
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(weights, bins=30, alpha=0.7)
plt.title('Distribution of IPW Weights')
plt.xlabel('Weight')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
treated_weights = weights[treatment.values == 1]
control_weights = weights[treatment.values == 0]
plt.hist(treated_weights, bins=20, alpha=0.7, label='Treated')
plt.hist(control_weights, bins=20, alpha=0.7, label='Control')
plt.title('Weights by Treatment Group')
plt.xlabel('Weight')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

## 5. Augmented IPW (AIPW) - Doubly Robust

AIPW combines G-computation and IPW to create a doubly robust estimator that is consistent if either the outcome model or propensity model is correctly specified.

In [ ]:
# Initialize AIPW estimator
aipw = AIPWEstimator(
    outcome_model='linear',
    propensity_model='logistic',
    bootstrap_samples=100
)

# Fit the estimator
print("Fitting AIPW estimator...")
aipw.fit(treatment, outcome, covariates)

# Estimate ATE
aipw_effect = aipw.estimate_ate()

print("\n=== AIPW Results ===")
print(f"Estimated ATE: ${aipw_effect.ate:.2f}")
print(f"95% Confidence Interval: [${aipw_effect.confidence_interval[0]:.2f}, ${aipw_effect.confidence_interval[1]:.2f}]")
print(f"Effect is significant: {aipw_effect.is_significant}")

print("\nDetailed Summary:")
print(aipw.summary())

## 6. Comparison of Methods

Let's compare all three methods and see how they perform.

In [ ]:
# Compile results
results = {
    'Method': ['G-Computation', 'IPW', 'AIPW', 'True ATE'],
    'ATE': [g_comp_effect.ate, ipw_effect.ate, aipw_effect.ate, 2.5],
    'CI_Lower': [g_comp_effect.confidence_interval[0], 
                 ipw_effect.confidence_interval[0], 
                 aipw_effect.confidence_interval[0], 
                 np.nan],
    'CI_Upper': [g_comp_effect.confidence_interval[1], 
                 ipw_effect.confidence_interval[1], 
                 aipw_effect.confidence_interval[1], 
                 np.nan],
    'Significant': [g_comp_effect.is_significant, 
                   ipw_effect.is_significant, 
                   aipw_effect.is_significant, 
                   True]
}

results_df = pd.DataFrame(results)
print("\n=== Comparison of Methods ===")
print(results_df.round(3))

# Visualize results
fig, ax = plt.subplots(figsize=(10, 6))

methods = ['G-Computation', 'IPW', 'AIPW']
ates = [g_comp_effect.ate, ipw_effect.ate, aipw_effect.ate]
cis_lower = [g_comp_effect.confidence_interval[0], 
             ipw_effect.confidence_interval[0], 
             aipw_effect.confidence_interval[0]]
cis_upper = [g_comp_effect.confidence_interval[1], 
             ipw_effect.confidence_interval[1], 
             aipw_effect.confidence_interval[1]]

# Plot point estimates and confidence intervals
x_pos = np.arange(len(methods))
ax.errorbar(x_pos, ates, 
           yerr=[np.array(ates) - np.array(cis_lower), 
                 np.array(cis_upper) - np.array(ates)], 
           fmt='o', capsize=5, capthick=2, markersize=8)

# Add true ATE line
ax.axhline(y=2.5, color='red', linestyle='--', alpha=0.7, label='True ATE')

ax.set_xlabel('Method')
ax.set_ylabel('Average Treatment Effect ($)')
ax.set_title('Comparison of Causal Inference Methods')
ax.set_xticks(x_pos)
ax.set_xticklabels(methods)
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

# Calculate bias and coverage
true_ate = 2.5
print("\n=== Method Performance ===")
for i, method in enumerate(methods):
    bias = ates[i] - true_ate
    covers_truth = cis_lower[i] <= true_ate <= cis_upper[i]
    ci_width = cis_upper[i] - cis_lower[i]
    
    print(f"{method}:")
    print(f"  Bias: ${bias:.3f}")
    print(f"  CI Width: ${ci_width:.3f}")
    print(f"  Covers Truth: {covers_truth}")
    print()

## 7. Diagnostics and Assumptions

Let's check key assumptions for causal inference.

In [ ]:
# Check covariate balance
print("=== Covariate Balance Assessment ===")
balance_result = check_covariate_balance(treatment, covariates)

balance_table = balance_result['balance_table']
print("\nStandardized Mean Differences:")
print(balance_table[['Variable', 'SMD', 'Balanced']].round(3))

# Check overlap/positivity
print("\n=== Overlap Assessment ===")
overlap_result = assess_positivity(treatment, covariates)

print(f"Common support satisfied: {overlap_result['common_support']}")
print(f"Propensity score range: [{overlap_result['propensity_range'][0]:.3f}, {overlap_result['propensity_range'][1]:.3f}]")

# Plot propensity score distributions
propensity_scores = overlap_result['propensity_scores']

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
treated_ps = propensity_scores[treatment.values == 1]
control_ps = propensity_scores[treatment.values == 0]

plt.hist(control_ps, bins=30, alpha=0.7, label='Control', density=True)
plt.hist(treated_ps, bins=30, alpha=0.7, label='Treated', density=True)
plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.title('Propensity Score Distribution by Treatment')
plt.legend()

plt.subplot(1, 2, 2)
smd_values = balance_table['SMD'].values
variables = balance_table['Variable'].values

colors = ['green' if abs(smd) < 0.1 else 'orange' if abs(smd) < 0.2 else 'red' for smd in smd_values]
plt.barh(variables, smd_values, color=colors)
plt.axvline(x=0.1, color='orange', linestyle='--', alpha=0.7, label='SMD = 0.1')
plt.axvline(x=-0.1, color='orange', linestyle='--', alpha=0.7)
plt.axvline(x=0.2, color='red', linestyle='--', alpha=0.7, label='SMD = 0.2')
plt.axvline(x=-0.2, color='red', linestyle='--', alpha=0.7)
plt.xlabel('Standardized Mean Difference')
plt.title('Covariate Balance')
plt.legend()

plt.tight_layout()
plt.show()

## 8. Interpretation and Conclusions

Let's interpret our results in the context of the marketing campaign.

In [ ]:
print("=== Marketing Campaign Analysis Summary ===")
print()
print("RESEARCH QUESTION:")
print("What is the causal effect of email marketing campaigns on customer purchase amounts?")
print()
print("KEY FINDINGS:")
print(f"• All three methods estimate a positive treatment effect")
print(f"• AIPW (doubly robust) estimate: ${aipw_effect.ate:.2f} increase in purchase amount")
print(f"• 95% confidence interval: [${aipw_effect.confidence_interval[0]:.2f}, ${aipw_effect.confidence_interval[1]:.2f}]")
print(f"• Effect is statistically significant: {aipw_effect.is_significant}")
print()
print("BUSINESS IMPLICATIONS:")
print(f"• Email campaigns increase average customer spending by approximately ${aipw_effect.ate:.2f}")
print(f"• This represents a meaningful lift for marketing ROI calculations")
print(f"• Results are robust across different estimation methods")
print()
print("METHODOLOGICAL NOTES:")
print(f"• Covariate balance: {'Good' if balance_result['overall_balance'] else 'Needs attention'}")
print(f"• Overlap assumption: {'Satisfied' if overlap_result['common_support'] else 'Violated'}")
print(f"• Sample size: {len(treatment.values)} customers")
print(f"• Treatment rate: {np.mean(treatment.values):.1%}")

# Calculate potential business impact
avg_customers_per_month = 10000
treatment_rate = 0.3  # 30% get email campaigns
monthly_treated = avg_customers_per_month * treatment_rate
monthly_lift = monthly_treated * aipw_effect.ate
annual_lift = monthly_lift * 12

print()
print("PROJECTED BUSINESS IMPACT:")
print(f"• Assuming {avg_customers_per_month:,} customers/month with {treatment_rate:.0%} treatment rate:")
print(f"• Monthly revenue lift: ${monthly_lift:,.0f}")
print(f"• Annual revenue lift: ${annual_lift:,.0f}")
print()
print("RECOMMENDATIONS:")
print("• Continue email marketing campaigns - they show clear positive impact")
print("• Consider expanding treatment to more customers if cost-effective")
print("• Monitor results over time to ensure effect persistence")
print("• Consider testing different email content or timing for optimization")

## Next Steps

This notebook covered the basics of causal inference for marketing applications. For more advanced topics, see:

1. **Marketing Use Cases** (`02_marketing_use_cases.ipynb`) - Specific marketing scenarios
2. **Advanced Diagnostics** (`03_advanced_diagnostics.ipynb`) - Detailed assumption checking
3. **Sensitivity Analysis** (`04_sensitivity_analysis.ipynb`) - Robustness testing
4. **Real Data Example** (`05_real_data_example.ipynb`) - Working with actual datasets

## Key Takeaways

1. **Multiple methods**: Use G-computation, IPW, and AIPW to triangulate results
2. **Diagnostics are crucial**: Always check balance, overlap, and other assumptions
3. **AIPW is robust**: Doubly robust methods provide insurance against model misspecification
4. **Business context matters**: Translate statistical results into actionable business insights
5. **Validation is key**: Use synthetic data to validate methods before applying to real data